In [ ]:
import cheb_tools as ct
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def mass_and_stiffness(n, quad_pts = 12):
    
    from numpy.polynomial.legendre import leggauss
    from scipy.interpolate import barycentric_interpolate
    
    D, x = ct.cheb(n-1)
    gx, gw = leggauss(quad_pts)
    
    M = np.zeros((n, n))
    S = np.zeros((n, n))
    for k in range(n):
        
        ek = np.zeros(n)
        ek[k] = 1
        Pk = barycentric_interpolate(x, ek, gx)
        
        for l in range(n):
            
            el = np.zeros(n)
            el[l] = 1
            Pl = barycentric_interpolate(x, el, gx)
            
            Del = D.dot(el)
            DPl = barycentric_interpolate(x, Del, gx)
            
            M[l,k] = np.sum(gw*Pk*Pl)
            S[l,k] = np.sum(gw*Pk*DPl)
            
    return M, S

In [ ]:
def test_mass_and_stiffness(n, quad_pts = 12):
    
    from sympy import cos, diff, integrate, N, pi, S, Symbol
    from sympy.polys.polyfuncs import interpolate
    from time import perf_counter
    
    X = Symbol('x')
    
    start = perf_counter()
    M, S = mass_and_stiffness(n, quad_pts)
    print('Time to approximate M and S = {:.3f}s'.format(perf_counter()-start))
    
    x = [-cos(pi*i/(n-1)) for i in range(n)]

    start = perf_counter()
    M_exact = np.zeros((n, n))
    S_exact = np.zeros((n, n))
    for k in range(n):
        ek = n*[0]
        ek[k] = 1
        pk = interpolate(list(zip(x,ek)), X)
        
        for l in range(n):
    
            el = n*[0]
            el[l] = 1
            pl = interpolate(list(zip(x,el)), X)
            dpl = diff(pl, X)
            
            M_exact[l,k] = N(integrate(pk*pl, (X, -1, 1)))
            S_exact[l,k] = N(integrate(pk*dpl, (X, -1, 1)))
    print('Time to find exact M and S  = {:.3f}s'.format(perf_counter()-start))

    print('Error in mass matrix        = {:.3e}'.format(np.max(np.abs(M-M_exact))))
    print('Error in stiffness matrix   = {:.3e}'.format(np.max(np.abs(S-S_exact))))
        
test_mass_and_stiffness(7)

In [ ]:
class DGStepper:
    
    def __init__(self, xl, xr, n_cells, m, f, f_hat):
        
        self.xl = xl
        self.xr = xr
        self.n_cells = n_cells
        self.m = m
        self.f = f
        self.f_hat = f_hat
        
        # endpoints of cells
        cell_x = np.linspace(xl, xr, n_cells+1)
        self.cell_x = cell_x
        
        # local grid
        loc_x = -np.cos(np.pi*np.arange(m)/(m-1))
        self.loc_x = loc_x
        
        # global locations of nodes
        x = np.concatenate([(cell_x[i]+cell_x[i+1]+loc_x*(cell_x[i+1]-cell_x[i]))/2 for i in range(n_cells)])
        self.x = x
        
        self.M, self.S = mass_and_stiffness(m)
        self.M *= (x[m-1]-x[0])/2
        
    def rhs(self, u, h, lflux=None, rflux=None):
        
        m = self.m
        f = self.f
        f_hat = self.f_hat
        S = self.S
        M = self.M
        n_cells = self.n_cells
        x = self.x
        
        # left and right limits
        um = np.zeros(n_cells+1)
        um[1:] = u[m-1::m]
        up = np.zeros(n_cells+1)
        up[:-1] = u[::m]
        
        flux = f_hat(um, up)
        if lflux is not None:
            flux[0] = lflux
        if rflux is not None:
            flux[-1] = rflux

        rhs = np.zeros(u.shape)
        for c in range(n_cells):

            # compute f inside cell
            F = f(u[c*m:(c+1)*m])

            # apply stiffness matrix
            SF = S.dot(F)

            # adjust with numerical fluxes
            SF[0]  += flux[c]
            SF[-1] -= flux[c+1]

            # solve for derivative
            rhs[c*m:(c+1)*m] = np.linalg.solve(M, SF) + h(x[c*m:(c+1)*m])
            
        return rhs
    
    def plot(self, u, color='b'):
        x = self.x
        m = self.m
        n_cells = self.n_cells
        for c in range(n_cells):
            plt.plot(x[c*m:(c+1)*m], u[c*m:(c+1)*m], color)


In [ ]:
def rhs_demo():
    a = 1.3
    f = lambda u : a*u
    h = lambda x : 0*x

    alpha = a

    f_hat = lambda um, up : 0.5*(f(um) + f(up) - alpha*(up - um))

    n_cells = 8
    m = 3
    xl, xr = -10, 10


    dg = DGStepper(xl, xr, n_cells, m, f, f_hat)
    
    U = lambda x: np.sin(0.8*x)
    u = U(dg.x)
    plt.figure(figsize=(8,6))
    xf = np.linspace(-10, 10, 1000)
    plt.plot(xf, -a*0.8*np.cos(0.8*xf), color='#cccccc')
    dg.plot(dg.rhs(u, h, lflux=f(U(xl))))

    
rhs_demo()

In [ ]:
def spectrum_demo():
    
    a = 0.9
    f = lambda u : a*u
    h = lambda x : 0*x

    alpha = a

    f_hat = lambda um, up : 0.5*(f(um) + f(up) - alpha*(up - um))

    n_cells = 10
    m = 3
    xl, xr = -10, 10

    CFL = 0.2
    
    dg = DGStepper(xl, xr, n_cells, m, f, f_hat)
    dx = dg.x[m-1]-dg.x[0]

    n_nodes = len(dg.x)
    A = np.zeros((n_nodes, n_nodes))
    for i in range(n_nodes):
        e = np.zeros(n_nodes)
        e[i] = 1
        A[:,i] = dg.rhs(e, h)
        
    l = np.linalg.eigvals(A)*CFL*dx/a
    plt.plot(np.real(l), np.imag(l), '.')
    mi = np.max(np.abs(np.imag(l)))
    plt.plot([0,0], [-mi,mi], '--')
    th = np.linspace(0, 2*np.pi, 1000)
    plt.plot(np.cos(th)-1, np.sin(th), 'r')
    plt.axis('equal')

    
spectrum_demo()

In [ ]:
def steady_demo():
    
    b = 0.9
    f = lambda u : u
    h = lambda x : b*np.cos(b*x)
    
    alpha = 1

    f_hat = lambda um, up : 0.5*(f(um) + f(up) - alpha*(up - um))

    n_cells = 10
    m = 2
    xl, xr = -10, 10
    
    CFL = 0.2

    dg = DGStepper(xl, xr, n_cells, m, f, f_hat)
    dx = dg.x[m-1]-dg.x[0]

    U = lambda x : np.sin(b*x)
    u = U(dg.x)
    dt = CFL*dx
    tf = 100
    Nt = int(np.ceil(tf/dt))
    dt = tf/Nt
    for it in range(Nt):
        u += dt*dg.rhs(u, h, lflux=f(U(xl)))
    
    xf = np.linspace(xl, xr, 1000)
    plt.plot(xf, U(xf), color='#dddddd')
    dg.plot(u)
    
    err = np.max(np.abs(u - U(dg.x)))
    print('error = {:.3e}'.format(err))
        
steady_demo()